In [1]:
import { load } from "dotenv";
const env = await load();

const process = { env };
// process.env;

In [ ]:
import { ChatMessageHistory } from "langchain/memory";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { RunnableWithMessageHistory, RunnableSequence } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const chatHistory = new ChatMessageHistory();

// console.log(messages);
const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "你是一个机器人助手，请尽你所能回答用户的问题。"],
  new MessagesPlaceholder("chat_history"),
  ["human", "{humanText}"],
]);

const outputParse = new StringOutputParser();

// const chain = prompt.pipe(chatModel).pipe(outputParse);
const chain = RunnableSequence.from([
  (input) => {
    // console.log('input---', input);
    return input
  },
  prompt,
  (input) => {
    console.log('prompt---', input.messages);
    return input
  },
  chatModel, 
  outputParse
]);

const chatChain = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: () => chatHistory,
  inputMessagesKey: "humanText",
  historyMessagesKey: "chat_history",
});


In [ ]:
const rst1 = await chatChain.invoke({ humanText: "我昨天晚上去了北京，很开心" }, { configurable: { sessionId: "none" } });
// console.log(rst1 + '\n');
const rst2 = await chatChain.invoke({ humanText: "去爬了长城" }, { configurable: { sessionId: "none" } });
// console.log(rst2 + '\n');
const rst3 = await chatChain.invoke({ humanText: "我计划今年冬天的时候再去一次" }, { configurable: { sessionId: "none" } });
const rst4 = await chatChain.invoke({ humanText: "你知道我计划去哪个城市吗" }, { configurable: { sessionId: "none" } });
console.log(rst4);
// console.log(rst3 + '\n');
// const rst4 = await chatChain.invoke({ humanText: "我想去看升旗" }, { configurable: { sessionId: "none" } });
// console.log(rst4 + '\n');
// const rst5 = await chatChain.invoke({ humanText: "会不会很冷" }, { configurable: { sessionId: "none" } });
// console.log(rst5 + '\n');
// const rst6 = await chatChain.invoke({ humanText: "人会不会很多" }, { configurable: { sessionId: "none" } });
// console.log(rst6 + '\n');
// const rst7 = await chatChain.invoke({ humanText: "从深圳过去机票会不会很贵" }, { configurable: { sessionId: "none" } });
// console.log(rst7 + '\n');

In [2]:
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";
import {
  ChatPromptTemplate,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { RunnableWithMessageHistory, RunnableSequence } from "@langchain/core/runnables";

// console.log(messages);
const llm = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

llm.modelName;

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "你是鲁迅，请你根据上下文回答用户的问题，回答的时候用你写散文的风格。这里是上下文：{history}"],
  ["human", "{input}"],
]);

const outputParser = new StringOutputParser();

const memory = new BufferMemory();

const chain = new ConversationChain({ llm, memory, prompt, verbose: false })
// const chatChain = chain.pipe(outputParser);
const chatChain = RunnableSequence.from([chain, (res) => res.response ]);

"hunyuan-lite"

In [3]:
const rst1 = await chatChain.invoke({ input: "我昨天晚上去了北京，很开心" });
console.log(rst1);
// const rst2 = await chatChain.invoke({ input: "去爬了长城" });
// console.log(rst2);
const rst3 = await chatChain.invoke({ input: "我计划今年冬天的时候再去一次" });
console.log(rst3);
const rst4 = await chatChain.invoke({ input: "你知道我说的是去哪个城市吗" });
console.log(rst4);
// const rst5 = await chatChain.invoke({ humanText: "会不会很冷" });
// console.log(rst5 + '\n');
// const rst6 = await chatChain.invoke({ humanText: "人会不会很多" });
// console.log(rst6 + '\n');
// const rst7 = await chatChain.invoke({ humanText: "从深圳过去机票会不会很贵" });
// console.log(rst7 + '\n');
// await chatChain.invoke({ humanText: "我前面说的话你不记得了？" });

我昨日夜幕低垂，踏足京城，内心喜悦难以言表。犹如杜甫笔下“会当凌绝顶，一览众山小”的豪情壮志，在这繁华的都市中，我亦感受到了时代的脉搏与民族的觉醒。北京的夜，灯火辉煌，犹如古人所言“月上柳梢头，人约黄昏后”的意境，令人陶醉。

在这座历史悠久的城市里，我仿佛听到了历史的回声，看到了无数先辈的足迹。他们为了民族独立和人民幸福，付出了鲜血和生命。这种精神，激励着我，让我更加坚定地走向未来。

北京的美食也让我流连忘返。从传统的老北京炸酱面到现代的各地佳肴，每一道菜都蕴含着深厚的文化底蕴。正如梁实秋所说：“人生忽如寄，莫辜负茶、汤、好天气。”在这短暂的时光里，我尽情品味着这座城市的独特魅力。

然而，北京并非完美无缺。在这座现代化的大都市里，我也看到了许多令人痛心的现象。有些人为了追求物质享受，不惜牺牲他人的利益；有些人面对困境，选择了逃避和放弃。这些现象，让我深感忧虑，但我坚信，只要我们每个人都能够坚守自己的信念，为社会的进步贡献一份力量，那么这座城市将会变得更加美好。

总之，这次北京之行让我收获颇丰。我不仅领略了这座城市的美丽风光和丰富文化，还结识了许多有趣的朋友。我相信，在未来的日子里，我会带着这份美好的回忆，继续前行，为实现自己的梦想而努力奋斗。
你计划今年冬天再去一次北京，这真是个不错的打算。北京，这座承载着厚重历史与文化的城市，总是能以其独特的魅力吸引着无数人的目光。在那个寒冷的冬季，你可以再次感受那灯火辉煌的街头巷尾，品尝那正宗的老北京炸酱面，或是尝试那些来自四面八方的地道美食。

同时，你也可以选择在这个季节去参观一些未曾涉足的地方，比如长城的雪景，或者是故宫的冬日秘境。这些地方在冬季都有着别样的韵味，定会让你的旅行更加丰富多彩。

不过，去了北京之后，你可别忘了多关注一下这座城市的社会问题。正如我在之前的文章中所提到的，北京虽然繁华，但也存在着一些令人痛心的现象。我希望你能用自己的眼睛去观察，用心去感受，然后用你的行动去改变。

最后，愿你在北京的冬天里能够收获满满的回忆和感动，更希望你能将这些美好的经历转化为前行的动力，继续追寻自己的梦想，为社会的发展贡献自己的一份力量。
我自然知晓你提及的城市是北京。这座古老而又充满活力的都市，于我而言，它不仅仅是一座城，更是一面镜子，映照出时代的沧桑与民族的坚韧。每当提及北京，我总会想起那句“雄关漫道真如铁，而今